<a href="https://colab.research.google.com/github/5harad/DPI-617/blob/main/labs/reasonable-suspicion-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Law, Order, and Algorithms**
## A statistical approach to assessing _reasonable suspicion_

**Getting started**

Before you start, create a copy of this Jupyter notebook in your own Google Drive by clicking `Copy to Drive` in the menubar. If you do not do this your work will not be saved! 

Remember to save your work frequently by pressing command-S or clicking File > Save in the menubar. 

We recommend completing this problem set in Google Chrome.

**Installing libraries**

As before, we'll start by loading the libraries that we'll use in this lab. But this time we'll need to install the `pROC` library first, which lets us compute error rates in predictive models. 

Run the cell below to install and load the libraries. It may take a minute or two to complete.

In [0]:
# install the library pROC, which lets us compute error rates
install.packages('pROC')

# load libraries
library(tidyverse)
library(scales)
library(pROC)

# Set some formatting options
options(digits = 3, repr.matrix.max.rows = 10, repr.matrix.max.cols = 100)
theme_set(theme_bw())

## Background

"Stop-and-frisk" is a police practice of temporarily detaining, questioning, and at times searching civilians on the street for weapons and other contraband.

In New York City, between 2003 and 2013, over 100,000 stops were recorded per year, with 685,724 stops recorded in a single year at the height of the practice in 2011. You can read more about this [here](https://en.wikipedia.org/wiki/Stop-and-frisk_in_New_York_City).

The vast majority of those stopped were African-American or Hispanic, raising concerns of racial bias. Further, there's concern that many stops were conducted without "reasonable suspicion" of criminal activity, in violation of the Fourth Amendment. 

In this lab, we will work with a sample of stops in NYC made between 2007 and 2013, recorded on a [UF-250 form](https://www.prisonlegalnews.org/news/publications/blank-uf-250-form-stop-question-and-frisk-report-worksheet-nypd-2016/), to explore a statistical approach to assessing potential Fourth Amendment violations outlined in the paper [Precinct or Prejudice? Understanding Racial Disparities in New York City's Stop-and-Frisk Policy](https://5harad.com/papers/stop-and-frisk.pdf) by Goel, Rao, and Shroff.

## The data

This week, we will use the same data that we used a couple of weeks ago: a sample of stops in NYC, recorded on a 
[UF-250 form][uf250_link].

Run the cell below to load the data. For simplicity, we'll work with a subset of the columns, and we'll limit to stops where there is no missing information (i.e., every variable has a value).

[uf250_link]: https://www.prisonlegalnews.org/media/publications/Blank%20UF-250%20Form%20-%20Stop%2C%20Question%20and%20Frisk%20Report%20Worksheet%2C%20NYPD%2C%202016.pdf
      

In [0]:
# Load data
fname <- "https://github.com/5harad/DPI-617/blob/main/data/sqf_sample.rds?raw=true"
stop_df <- readRDS(url(fname))

# Select the columns we'll use in the lab today, and exclude missing observations
stop_df <- stop_df %>%
  select('found_weapon', 'frisked', 'suspected_crime', 'precinct',
         'month', 'day', 'year',
         'inside_outside', 'location_housing',
         'radio_run', 'officer_uniform', 'observation_period',
         starts_with('subject_'), -subject_hispanic, -subject_dob,
         starts_with('stopped_bc_'),
         starts_with('additional_')) %>%
  na.omit()

stop_df

## Analysis

For this lab, we will focus on stops for suspicion of criminal possession of a weapon (CPW).

###Exercise: Data filtering

In [0]:
# Your code here!
# Create a data frame named stops_cpw with only cpw stops,
# as indicated by column suspected_crime


One might say that a CPW stop was _ex post_ justified if a weapon was ultimately recovered. To gauge whether such a stop was _ex ante_ justified, we will use the available data to build a risk assessment model, allowing us to estimate the likelihood a stopped individual is carrying a weapon. We can then examine the model-estimated risk scores to see whether people were stopped even though the available evidence suggests that they are unlikely to be carrying a weapon, potentially in violation of the Fourth Amendment. We will look at each of these steps — estimating risk and examining scores — in turn below.

### Estimating risk

For an individual $i$, let $X_i$ be all the information we know about the
individual prior to making the stop decision, and define


$$
Y_i =
\begin{cases}
1 & \textrm{if a weapon is found on individual } i \\
0 & \textrm{otherwise}
\end{cases}
$$


Then, the _risk_ we are interested in measuring can be written as

$$
r_i = \Pr(Y_i = 1 ~|~ X_i).
$$



In other words, we wish to estimate the probability that we will recover a 
weapon from an individual, characterized by covariates $X_i$, if we were to 
stop (and frisk) them.

One way to estimate this probability is to fit a logistic regression on the
subset of individuals who were stopped, to predict whether a weapon will be
found, using all the covariates that an officer observed prior to conducting the
stop. 

Don't worry if you haven't heard of logistic regression before! We'll talk about
it more in later lectures; for now, we'll treat logistic regression as a black-box
that spits out an estimated probability of finding a weapon on someone matching
a certain set of criteria (or _features_) that we specify.

To fit and evaluate our risk model, we split the data into two parts: a _training_ set and a _test_ set.
We fit (or _train_) our logistic regression model using the training set, and then use the fitted model to generate predictions on the test set. This splitting of the data helps to ensure that the model isn't able to "cheat" when making its predictions.

In [0]:
# Split train-test set
stops_train_cpw <- stops_cpw %>%
  filter(year %in% 2009:2010)

stops_test_cpw <- stops_cpw %>%
  filter(year %in% 2011:2012)

We'll start be estimating risk based on whether the officer reported seeing a "bulge" (indicative of carrying a weapon) and whether the officer indicated that the stop occurred in a "high-crime" neighborhood. We train and make predictions with the logistic regression model as follows:

In [0]:
# We start by specifying which risk factors to include in the model
features <- c('stopped_bc_bulge', 'additional_highcrime')

# We will use all covariates listed in "features" to predict the outcome "found_weapon"
glm_formula <- reformulate(features, response="found_weapon")
glm_fit <- glm(glm_formula, data = stops_train_cpw, family = binomial())

Using the model we trained above, we'll make predictions on the test.

At the end of this process, we get a vector of predictions that indicates how likely each person in the test set is to be carrying a weapon, based on the two specified risk factors.

In [0]:
# Make predictions on the test set
predictions <- predict(glm_fit, newdata = stops_test_cpw, type = "response")

In [0]:
head(predictions)

To see where these predictions are coming from, we can check how much weight the model is putting on each of the factors, displayed in the `estimate` column in the table below. Based on the historical data, the model believes that people stopped because of a reported "bulge" are at higher risk of carrying a weapon (since the estimate is positive). Further, people stopped because they are in a "high crime" neighborhood are at _lower_ risk of carrying a weapon, compared to those without this factor reported as a basis for the stop (since the estimate is negative).

In [0]:
summary(glm_fit)

### Exercise: Model interpretation
Why do you think the "high crime" rationale is associated with _lower_ risk of carrying a weapon in the model above?

Now we'll quantify how well this risk model performs.
There are many ways to evaluate a model's performance.
For a binary classification task like the one we are working on (predicting whether or not someone is carrying a weapon), a common performance metric is [AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). Given a random pair of people in the dataset, exactly one of whom is carrying a weapon, the AUC is the probability that the model will correctly identify the person with the weapon. A perfect binary prediction model will thus give an AUC of 1 while a model that is making random guesses will produce an AUC of 0.5.

To compute the AUC, we'll use the `auc` function (from the `pROC` library that we installed earlier). The `auc` function requires two inputs: (1) the true outcomes (i.e., whether or not someone was carrying a weapon; and (2) the predictions.

The true outcomes (i.e., whether someone was carrying a weapon) is stored in the `found_weapon` column of the `stops_test_cpw` data frame. To pull out the `found_weapon` column as a stand-alone vector, we use the `$` operator.

In [0]:
stops_test_cpw$found_weapon

Now we can compute the AUC by providing the `auc` function the vector of true outcomes and the vector of predictions.

In [0]:
auc(stops_test_cpw$found_weapon, predictions)

### Exercise: Interpreting model fit
Discuss the AUC of the risk model above. How well would you say the model is doing?

### Exercise: Building a better predictive model

In the above model, we used `stopped_bc_bulge` and `additional_highcrime` as the only risk factors used to predict whether an individual has a weapon. Explore incorporating additional factors from the dataset to build a more accurate model. To do this, start with the model-building code above, and then add additional factors to the `features` vector.

In [0]:
# Your code here!
# Build a better model by including more features


You can run the code below to compute the AUC of your new predictions

In [0]:
auc(stops_test_cpw$found_weapon, predictions)

### Exercise: Interpreting risk scores

Discuss your risk model. 
Do you think it gives an accurate estimate of the likelihood someone has a weapon, 
based on information observed by the officer?

### Assessing 4A violations

We'll now use our model estimates to assess potential Fourth Amendment violations. To start, we'll compute the fraction of CPW stops that have less than 1% _ex ante_ chance of recovering a weapon.


### Exercise: Calculate the proportion of stops that have low probability of "success".

Step 1. Starting with your `predictions`, create a binary vector (with `TRUE` and `FALSE` values) that indicates whether each stop had less than a 1% _ex ante_ chance of recovering a weapon.

Step 2. With this binary vector, compute the proportion of values that are `TRUE`. You can use the `length` function to compute the length of a vector (i.e, the number of entries in it).

Once you compute the fraction of stops that have low likelihood of recovering a weapon, discuss the result. Do you think this provides evidence of Fourth Amendment violations?

In [0]:
# Your code here!


Now we'll repeat this exercise for every threshold (e.g., 0.1% and 2%). 
(This is called the [cumulative distirbution function (CDF)](https://en.wikipedia.org/wiki/Cumulative_distribution_function) of the risk distribution.)

In [0]:
stops_test_cpw %>%
  mutate(found_weapon_p = predictions) %>%
  ggplot(aes(found_weapon_p)) + 
  stat_ecdf(pad=FALSE) +
  scale_x_continuous(
    'Ex ante probabilty of finding a weapon', 
    limits = c(0,0.1),
    labels=percent_format()) +
  scale_y_continuous(
    'Proportion of stops below the threshold', 
    limits = c(0,1),
    labels=percent_format()
  )


### Exercise: Interpreting the result

Discuss the result above. What do you think the plot above tells us about Fourth Amendment violations?

Now let's generate a version of the plot above disaggregated by race, for those categorized in the dataset as `Black`, `Hispanic`, or `white`.

In [0]:
stops_test_cpw %>%
  mutate(found_weapon_p = predictions, 
         subject_race = str_to_title(subject_race)) %>%
  filter(subject_race %in% c('White','Black','Hispanic')) %>%
  ggplot(aes(found_weapon_p, color=subject_race)) + 
  stat_ecdf(pad=FALSE) +
  scale_x_continuous(
    'Ex ante probabilty of finding a weapon', 
    limits = c(0,0.1),
    labels=percent_format()) +
  scale_y_continuous(
    'Proportion of stops below the threshold', 
    limits = c(0,1),
    labels=percent_format()
  ) + 
  guides(color=guide_legend(title="Race"))


### Exercise: Interpreting the result

Discuss the result above. What do you think the plot above tells us about Fourth Amendment violations? What does it tell us about discrimination?